In [286]:
import numpy as np
from scipy.optimize import minimize
import pandas as pd

In [216]:
#stockPool is a huge array, good memory??

stockPool = np.random.random((100,8192))

In [527]:
class portfolio:
    
    def __init__(self,name,size):
        self.stocks = np.random.choice(np.arange(20), size=size, replace=False)
        self.weights = dict.fromkeys(self.stocks,0) #init with a sharpe function, put in dictionary for easy change
        self.orders = np.zeros(size)
        self.portfID = str(name)
    
    def order(self):
        """
        returns the orders to be added to the broker dataframe
        """
        
        orderList = pd.DataFrame({"portfolio":self.portfID,"stock":self.stocks, "order": self.orders})
        
        return orderList
        

In [219]:
def sharpe(weights,stocks, time):
    """
    Input the stock ID numbers and calculate the sharpe ratio based on time
    """
    Rp = 0
    Rf = 0.001
    var = 0
    for i,j in enumerate(stocks): 
        Rp += weights[i]*np.mean(stockPool[j][:time]) #expected return up to point in time
        var += weights[i]*weights[i]*np.var(stockPool[j][:time]) #variance return up to point in time
    stdp = np.sqrt(var)
    
    return -(Rp-Rf)/stdp
    
    

In [224]:
#opt= minimize(sharpe, a.weights, args=(a.stocks,6), method='Nelder-Mead', tol=1e-6)['x'] 
# minimize from steady state based on fBM

In [528]:
# to generate overlapping portfolios, scan through the stock list with various overlapping amounts. 
# for trades, generate an order book for each portfolio with the stocks they want to buy or sell, these are
# then matched to the closest order from another portfolio, a trade is agreed upon once the sum of their 
# sharpe ratios are maxamized by this trade

# INITIALIZE PORTFOLIO FUNCTION
a = portfolio('a',10)
aopt = [-3,-2,-1,-2,-5,-1,-1,-1,-1,-3] 
a.orders = np.asarray(aopt) - list(a.weights.values())
alist = a.order()

b = portfolio("b",8)
bopt = [3,4,5,6,7,8,2,0]
b.orders = np.asarray(bopt) - list(b.weights.values())
blist = b.order()


c = portfolio("c",8)
copt = [3,4,5,6,7,8,2,0]
c.orders = np.asarray(copt) - list(c.weights.values())
clist = c.order()


d = portfolio("d",8)
dopt = [-3,-4,-5,6,-7,8,2,-2]
d.orders = np.asarray(dopt) - list(d.weights.values())
dlist = d.order()

In [529]:
broker = pd.concat([alist,blist,clist,dlist])
broker = broker[broker.order!=0] # remove all null orders
potMatch = broker[broker.groupby('stock').stock.transform(len) > 1] # removes all single orders

In [531]:
potMatch

,portfolio,stock,order
0,a,18,-3
2,a,19,-1
4,a,7,-5
5,a,0,-1
6,a,6,-1
7,a,11,-1
8,a,2,-1
0,b,6,3
1,b,3,4
2,b,18,5


In [608]:
orderList = pd.DataFrame()

for stock in (potMatch.stock.unique()): 
    # go through each stock and try to fill orders
    stockSearch = (potMatch[potMatch.stock == stock])
    buy = stockSearch[stockSearch.order > 0]
    sell = stockSearch[stockSearch.order < 0]
    if len(buy) == 0 or len(sell) == 0:
        print("No match")
        continue
    buy = buy.sort_values(by="order",ascending=False)
    sell = sell.sort_values(by="order",ascending=True)
    buy.reset_index(inplace=True,drop=True)
    sell.reset_index(inplace=True,drop=True)
    #print("BUY")
    #print(buy)
    #print("SELL")
    #print(sell)

    while len(buy) != 0 and len(sell) != 0:
        # iterate until no more matches
        for sID in range(len(sell)):
            s = sell.iloc[sID].order #selling amount
            if -s in buy.order.values:
                # see if there is a matching order to easily fill right away
                bMatch = buy[buy.order.isin([-s])].iloc[0]

                sale = pd.DataFrame({"stock":stock, "seller": sell.iloc[sID].portfolio, 
                                     "buyer": bMatch.portfolio, "volume": -s, "tradeID": 123},index=['t'])

                orderList = pd.concat([orderList,sale])

                print("sale made")

            else:
                # algorithm that matches s with a b and updates sell and buy rows, take min and subtract until 0
                
            buy = buy[buy.order!=0]
            sell = sell[sell.order!=0]

                

sale made
No match


In [609]:
orderList.loc['t (fill in)']

stock        6
seller       d
buyer        b
volume       3
tradeID    123
Name: t (fill in), dtype: object

In [571]:
orderList.concat({"portfolio":self.portfID,"stock":self.stocks, "order": self.orders})

AttributeError: 'DataFrame' object has no attribute 'concat'

In [582]:
potMatch[potMatch.order.isin([-1])].iloc[0].portfolio

'a'

In [565]:
-8 in potMatch.order.values

False

In [562]:
potMatch.order.values

array([-3, -1, -5, -1, -1, -1, -1,  3,  4,  5,  8,  3,  4,  5,  6,  7,  8,
       -3, -4, -5,  6, -7,  8,  2, -2])

In [566]:
len(potMatch)

25

In [570]:
for i in potMatch:
    print(i.order)

AttributeError: 'str' object has no attribute 'order'